In [1]:
!git clone https://github.com/peppermenta/faceHack

Cloning into 'faceHack'...
remote: Enumerating objects: 6470, done.
remote: Counting objects: 100% (6470/6470), done.
remote: Compressing objects: 100% (6467/6467), done.
remote: Total 6470 (delta 7), reused 6457 (delta 1), pack-reused 0
Receiving objects: 100% (6470/6470), 61.98 MiB | 39.72 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!pip3 install -r faceHack/requirements.txt

In [3]:
import torch
from PIL import Image
import numpy as np
import os      
import torchvision         

class FaceDataset(torch.utils.data.Dataset):
  def __init__(self,train=True,type='normal',transform=None):
    super().__init__()
    if(train):
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','train.csv'),delimiter=',',dtype=None,encoding='utf-8')
    else:
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','test.csv'),delimiter=',',dtype=None,encoding='utf-8')

    self.type=type
    self.transform = transform

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index):
    img_name = self.data[index][0]
    X = Image.open(os.path.join('faceHack','dataset','happy_images','{}.jpg'.format(img_name)))
    label = self.data[index][1]

    if(self.type=='normal'):
      if(label=='NOT smile'):
        y = 0
      else:
        y = 1
    else:
      if(label=='positive smile'):
        y = 1
      else:
        y = 0

    if(self.transform):
      X = self.transform(X)
    return X,y


In [4]:
faceTransform = torchvision.transforms.Compose([
  torchvision.transforms.Resize(224),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.ToTensor(),
])

In [16]:
device = torch.device('cuda')
print(device)
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)

# for param in model.parameters():
#   param.requires_grad = False

in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features=in_features,out_features=2)
model = model.to(device)


trainDataset = FaceDataset(transform=faceTransform)
testDataset = FaceDataset(train=False,transform=faceTransform)
trainLoader = torch.utils.data.DataLoader(trainDataset,batch_size=32,shuffle=True,num_workers=2)
testLoader = torch.utils.data.DataLoader(testDataset,batch_size=32,shuffle=False,num_workers=2)

cuda


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


In [20]:
model.train()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.00001)
epochs = 5

loss_hist = []

for epoch in range(epochs):
  tot_loss = 0
  correct = 0
  for i,data in enumerate(trainLoader):
    X,y = data
    X = X.to(device)
    y = y.to(device)

    out = model(X)

    print(out)
    pred = torch.max(out,dim=1)
    correct += (pred==y).sum().item()

    loss = criterion(out,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    tot_loss += loss.item()
    if(i%2==0):
      print('Epoch:({},{}): Loss={}'.format(epoch,i,loss.item()))

  loss_hist.append(tot_loss)
  print('Accuracy is {}'.format(correct/len(trainDataset)))


import matplotlib.pyplot as plt

plt.plot([i for i in range(epochs)],loss_hist)


RuntimeError: ignored

In [ ]:
model.eval()